In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlencode
import urllib.parse
from copy import copy
import requests
import re
import json
import urllib.request

## Scrape event information for a driver

In [2]:
def events_for_a_driver(my_url):
    
    # Load races on url
    raw_html = requests.get(my_url).content
    soup_doc = BeautifulSoup(raw_html, "html.parser")
    events = soup_doc.find_all("div", class_="profile-start-line d-flex justify-content-start w-100 lh-120 py-1 flex-wrap")
    races = []
    
    # Find all WRC races
    events = [e for e in events if e.span.text == ' WRC']

    # List out all informations
    
    for e in events:
        race = {}
        race['Driver'] = soup_doc.find("h4").a.text.strip()
        race['Name'] = e.find("a", title="Show entry info and stats").text
        URL = e.find("a", title="Show entry info and stats")['href']
        race['URL'] = URL
    # Check the URL
        print(URL)
        id1 = re.findall("/entryinfo/(\d+)",URL)[0]
        id2 = re.findall("/entryinfo/.+/(\d+)",URL)[0]
        Year = re.findall("/entryinfo/.+-(\d\d\d\d)/\d+/",URL)[0]
        race['Year'] = Year
        
        
        try:
            rank = e.find("span", class_="fs-09 font-weight-normal text-secondary").text
            ra = re.findall("(\d).",rank)[0]
            race['Race_Result'] = ra
            race['DNG_Reason'] = "Null"
        except:
            ra = e.find("div", class_="profile-start-ret text-danger font-weight-bold")
            race['Race_Result'] = "DNF"
            race['DNG_Reason'] = ra
        
        race['Php_url'] = "https://www.ewrc-results.com/inc/json_driverinfo.php?event="+id1+"&entry="+id2+"&sct=&cat=&lg=en"
        
        print("Php found" + race['Php_url'])
        
        x = urllib.request.urlopen(race['Php_url'])
        raw_data = x.read()
        encoding = x.info().get_content_charset('utf8')  # JSON default
        ssresult = json.loads(raw_data.decode(encoding))
        print(ssresult, len(ssresult['stages'][0][1]))

        
        for idx, pos in enumerate(ssresult['pos'][0][1]):
            
            stage = race.copy()
            
            stage['p_overall'] = pos      
            stage['stage'] = ssresult['stages'][0][1][idx]
            stage['ssname'] = ssresult['sslist'][idx]
            races.append(stage)
            # print(ssresult['pos'][0][1])
        
        
        print("Php info scraped.")
        
        
    return races



In [3]:
races = events_for_a_driver('https://www.ewrc-results.com/profile/4298-thierry-neuville/')

/entryinfo/79247-rally-sweden-2023/3608291/
Php foundhttps://www.ewrc-results.com/inc/json_driverinfo.php?event=79247&entry=3608291&sct=&cat=&lg=en
{'pos': [['Overall', [5, 6, 6, 7, 6, 6, 6, 6, 6, 6, 6, 5, 4, 3, 3, 3, 2, 3]]], 'stages': [['Stage results', [5, 5, 6, 7, 6, 5, 5, 4, 1, 3, 2, 3, 1, 1, 1, 3, 1, 6]]], 'sslist': ['SS1', 'SS2', 'SS3', 'SS4', 'SS5', 'SS6', 'SS7', 'SS8', 'SS9', 'SS10', 'SS11', 'SS12', 'SS13', 'SS14', 'SS15', 'SS16', 'SS17', 'SS18'], 'colors': ['#e2431e', '#f1ca3a']} 18
Php info scraped.
/entryinfo/77787-rallye-automobile-monte-carlo-2023/3560761/
Php foundhttps://www.ewrc-results.com/inc/json_driverinfo.php?event=77787&entry=3560761&sct=&cat=&lg=en
{'pos': [['Overall', [3, 4, 4, 3, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]]], 'stages': [['Stage results', [3, 4, 4, 3, 3, 5, 4, 2, 4, 4, 4, 1, 1, 2, 4, 3, 4, 4]]], 'sslist': ['SS1', 'SS2', 'SS3', 'SS4', 'SS5', 'SS6', 'SS7', 'SS8', 'SS9', 'SS10', 'SS11', 'SS12', 'SS13', 'SS14', 'SS15', 'SS16', 'SS17', 'SS18'], 'color

{'pos': [['Overall', [4, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4]]], 'stages': [['Stage results', [4, 3, 3, 3, 3, 1, 4, 1, 9, 2, 4, 3, 2, 1, 1, 1]]], 'sslist': ['SS1', 'SS2', 'SS3', 'SS4', 'SS5', 'SS6', 'SS7', 'SS8', 'SS9', 'SS10', 'SS11', 'SS12', 'SS13', 'SS14', 'SS15', 'SS16'], 'colors': ['#e2431e', '#f1ca3a']} 16
Php info scraped.
/entryinfo/68118-rallyracc-catalunya-costa-daurada-2021/3126187/
Php foundhttps://www.ewrc-results.com/inc/json_driverinfo.php?event=68118&entry=3126187&sct=&cat=&lg=en
{'pos': [['Overall', [2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]], 'stages': [['Stage results', [2, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 2, 3, 2]]], 'sslist': ['SS1', 'SS2', 'SS3', 'SS4', 'SS5', 'SS6', 'SS7', 'SS8', 'SS9', 'SS10', 'SS11', 'SS12', 'SS13', 'SS14', 'SS15', 'SS16', 'SS17'], 'colors': ['#e2431e', '#f1ca3a']} 17
Php info scraped.
/entryinfo/68115-secto-rally-finland-2021/3118316/
Php foundhttps://www.ewrc-results.com/inc/json_driverinfo.php?event=68115&entry=311831

{'pos': [['Overall', [5, 6, 6, 6, 6, 6, 6, 6, 6]]], 'stages': [['Stage results', [5, 7, 8, 7, 4, 4, 6, 1, 2]]], 'sslist': ['SS2', 'SS3', 'SS4', 'SS5', 'SS6', 'SS7', 'SS8', 'SS9', 'SS11'], 'colors': ['#e2431e', '#f1ca3a']} 9
Php info scraped.
/entryinfo/59972-rallye-automobile-monte-carlo-2020/2465681/
Php foundhttps://www.ewrc-results.com/inc/json_driverinfo.php?event=59972&entry=2465681&sct=&cat=&lg=en
{'pos': [['Overall', [4, 1, 1, 2, 2, 3, 3, 3, 3, 3, 3, 3, 2, 1, 1, 1]]], 'stages': [['Stage results', [4, 1, 6, 3, 3, 2, 3, 1, 1, 3, 1, 1, 1, 1, 1, 1]]], 'sslist': ['SS1', 'SS2', 'SS3', 'SS4', 'SS5', 'SS6', 'SS7', 'SS8', 'SS9', 'SS10', 'SS11', 'SS12', 'SS13', 'SS14', 'SS15', 'SS16'], 'colors': ['#e2431e', '#f1ca3a']} 16
Php info scraped.
/entryinfo/54472-rallyracc-catalunya-costa-daurada-2019/2386038/
Php foundhttps://www.ewrc-results.com/inc/json_driverinfo.php?event=54472&entry=2386038&sct=&cat=&lg=en
{'pos': [['Overall', [3, 1, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]], 'stages'

{'pos': [['Overall', [2, 3, 7, 8, 8, 9, 9, 8, 6, 6, 6, 5, 5, 5, 4, 3, 4]]], 'stages': [['Stage results', [2, 7, 8, 9, 9, 10, 8, 5, 3, 1, 2, 1, 8, 3, 3, 4, 6]]], 'sslist': ['SS1', 'SS2', 'SS3', 'SS4', 'SS5', 'SS6', 'SS7', 'SS9', 'SS10', 'SS11', 'SS12', 'SS13', 'SS14', 'SS15', 'SS16', 'SS17', 'SS18'], 'colors': ['#e2431e', '#f1ca3a']} 17
Php info scraped.
/entryinfo/43469-dayinsure-wales-rally-gb-2018/1970353/
Php foundhttps://www.ewrc-results.com/inc/json_driverinfo.php?event=43469&entry=1970353&sct=&cat=&lg=en
{'pos': [['Overall', [2, 3, 6, 6, 4, 5, 4, 2, 2, 2, 8, 8, 9, 9, 9, 8, 8, 8, 8, 8, 7, 5, 5]]], 'stages': [['Stage results', [2, 5, 6, 6, 1, 7, 6, 2, 1, 8, 12, 10, 9, 5, 7, 5, 7, 3, 7, 4, 3, 3, 2]]], 'sslist': ['SS1', 'SS2', 'SS3', 'SS4', 'SS5', 'SS6', 'SS7', 'SS8', 'SS9', 'SS10', 'SS11', 'SS12', 'SS13', 'SS14', 'SS15', 'SS16', 'SS17', 'SS18', 'SS19', 'SS20', 'SS21', 'SS22', 'SS23'], 'colors': ['#e2431e', '#f1ca3a']} 23
Php info scraped.
/entryinfo/44264-rally-turkey-marmaris-2018/

{'pos': [['Overall', [2, 10, 12, 9, 9, 9, 9, 9, 9, 9, 8, 8, 8, 8, 8, 7, 7, 7, 6, 6, 6, 6, 6, 6, 6]]], 'stages': [['Stage results', [2, 12, 12, 9, 11, 10, 10, 9, 4, 8, 9, 7, 1, 9, 9, 6, 6, 4, 3, 8, 4, 9, 12, 6, 3]]], 'sslist': ['SS1', 'SS2', 'SS3', 'SS4', 'SS5', 'SS6', 'SS7', 'SS8', 'SS9', 'SS10', 'SS11', 'SS12', 'SS13', 'SS14', 'SS15', 'SS16', 'SS17', 'SS18', 'SS19', 'SS20', 'SS21', 'SS22', 'SS23', 'SS24', 'SS25'], 'colors': ['#e2431e', '#f1ca3a']} 25
Php info scraped.
/entryinfo/36431-orlen-rally-poland-2017/1489966/
Php foundhttps://www.ewrc-results.com/inc/json_driverinfo.php?event=36431&entry=1489966&sct=&cat=&lg=en
{'pos': [['Overall', [2, 1, 3, 4, 2, 3, 3, 1, 1, 1, 2, 2, 1, 1, 2, 2, 1, 1, 2, 1, 1, 1]]], 'stages': [['Stage results', [2, 1, 4, 4, 1, 8, 1, 1, 3, 2, 4, 2, 1, 1, 12, 2, 1, 2, 2, 1, 1, 5]]], 'sslist': ['SS1', 'SS2', 'SS3', 'SS4', 'SS5', 'SS7', 'SS8', 'SS9', 'SS10', 'SS11', 'SS12', 'SS13', 'SS14', 'SS15', 'SS16', 'SS17', 'SS18', 'SS19', 'SS20', 'SS21', 'SS22', 'SS23'], '

{'pos': [['Overall', [6, 5, 3, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]], 'stages': [['Stage results', [6, 5, 2, 1, 1, 4, 1, 1, 1, 3, 6, 4, 1, 2, 1, 1, 2, 1, 5]]], 'sslist': ['SS1', 'SS2', 'SS3', 'SS4', 'SS5', 'SS6', 'SS7', 'SS8', 'SS9', 'SS10', 'SS11', 'SS12', 'SS13', 'SS14', 'SS15', 'SS16', 'SS17', 'SS18', 'SS19'], 'colors': ['#e2431e', '#f1ca3a']} 19
Php info scraped.
/entryinfo/27503-vodafone-rally-de-portugal-2016/1157283/
Php foundhttps://www.ewrc-results.com/inc/json_driverinfo.php?event=27503&entry=1157283&sct=&cat=&lg=en
{'pos': [['Overall', [2, 11, 8, 8, 5, 5, 5, 5, 5, 5, 18, 25, 33, 35, 38, 35, 34, 29, 29]]], 'stages': [['Stage results', [2, 12, 4, 6, 4, 4, 7, 1, 1, 5, 62, 57, 50, 42, 43, 7, 4, 7, 5]]], 'sslist': ['SS1', 'SS2', 'SS3', 'SS4', 'SS5', 'SS6', 'SS7', 'SS8', 'SS9', 'SS10', 'SS11', 'SS12', 'SS13', 'SS14', 'SS15', 'SS16', 'SS17', 'SS18', 'SS19'], 'colors': ['#e2431e', '#f1ca3a']} 19
Php info scraped.
/entryinfo/27493-ypf-rally-argentina-2016/1124983/
Php fou

{'pos': [['Overall', [1, 2, 2, 2, 2, 2, 2, 12, 16, 19, 17, 16, 15, 15, 12, 12, 11, 11, 10, 10, 8]]], 'stages': [['Stage results', [1, 4, 2, 5, 1, 6, 5, 19, 21, 22, 8, 5, 4, 5, 5, 4, 4, 4, 4, 1, 3]]], 'sslist': ['SS1', 'SS2', 'SS3', 'SS4', 'SS5', 'SS6', 'SS7', 'SS8', 'SS9', 'SS10', 'SS11', 'SS12', 'SS13', 'SS14', 'SS15', 'SS16', 'SS17', 'SS18', 'SS19', 'SS20', 'SS21'], 'colors': ['#e2431e', '#f1ca3a']} 21
Php info scraped.
/entryinfo/19120-rally-sweden-2015/743381/
Php foundhttps://www.ewrc-results.com/inc/json_driverinfo.php?event=19120&entry=743381&sct=&cat=&lg=en
{'pos': [['Overall', [4, 6, 6, 5, 5, 5, 5, 5, 2, 2, 2, 3, 3, 3, 2, 2, 2, 1, 2, 3, 2]]], 'stages': [['Stage results', [4, 6, 6, 5, 5, 4, 5, 3, 1, 3, 5, 5, 4, 2, 1, 6, 10, 1, 3, 4, 2]]], 'sslist': ['SS1', 'SS2', 'SS3', 'SS4', 'SS5', 'SS6', 'SS7', 'SS8', 'SS9', 'SS10', 'SS11', 'SS12', 'SS13', 'SS14', 'SS15', 'SS16', 'SS17', 'SS18', 'SS19', 'SS20', 'SS21'], 'colors': ['#e2431e', '#f1ca3a']} 21
Php info scraped.
/entryinfo/18216-

{'pos': [['Overall', [3, 3, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]]], 'stages': [['Stage results', [3, 3, 1, 2, 6, 7, 4, 7, 3, 4, 3, 4, 3, 5, 3, 1, 4, 6, 1, 6, 4, 4]]], 'sslist': ['SS1', 'SS2', 'SS3', 'SS4', 'SS5', 'SS6', 'SS7', 'SS8', 'SS9', 'SS10', 'SS11', 'SS12', 'SS13', 'SS14', 'SS15', 'SS16', 'SS17', 'SS18', 'SS19', 'SS20', 'SS21', 'SS22'], 'colors': ['#e2431e', '#f1ca3a']} 22
Php info scraped.
/entryinfo/6542-rallyracc-catalunya-costa-daurada-2013/416152/
Php foundhttps://www.ewrc-results.com/inc/json_driverinfo.php?event=6542&entry=416152&sct=&cat=&lg=en
{'pos': [['Overall', [4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 5, 5, 5, 4, 4]]], 'stages': [['Stage results', [4, 4, 5, 6, 5, 3, 4, 1, 2, 5, 9, 8, 2, 4, 3]]], 'sslist': ['SS1', 'SS2', 'SS3', 'SS4', 'SS5', 'SS6', 'SS7', 'SS8', 'SS9', 'SS10', 'SS11', 'SS12', 'SS13', 'SS14', 'SS15'], 'colors': ['#e2431e', '#f1ca3a']} 15
Php info scraped.
/entryinfo/6541-rallye-de-france-alsace-2013/397910/
Php foundhttps://www.ewrc-result

{'pos': [['Overall', [8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 7, 7, 7]]], 'stages': [['Stage results', [8, 7, 8, 8, 8, 8, 8, 8, 8, 8, 6, 8, 5, 6, 6, 6, 6, 6, 6]]], 'sslist': ['SS1', 'SS2', 'SS3', 'SS4', 'SS5', 'SS6', 'SS7', 'SS8', 'SS9', 'SS10', 'SS11', 'SS12', 'SS13', 'SS14', 'SS15', 'SS16', 'SS17', 'SS18', 'SS19'], 'colors': ['#e2431e', '#f1ca3a']} 19
Php info scraped.
/entryinfo/3497-adac-rallye-deutschland-2012/167085/
Php foundhttps://www.ewrc-results.com/inc/json_driverinfo.php?event=3497&entry=167085&sct=&cat=&lg=en
{'pos': [['Overall', [5, 4, 4, 4, 4, 4, 4, 4, 9, 15, 15, 14, 13, 13, 12]]], 'stages': [['Stage results', [5, 4, 6, 3, 5, 4, 5, 7, 38, 51, 52, 34, 7, 17, 6]]], 'sslist': ['SS1', 'SS2', 'SS3', 'SS4', 'SS5', 'SS6', 'SS7', 'SS8', 'SS9', 'SS10', 'SS11', 'SS12', 'SS13', 'SS14', 'SS15'], 'colors': ['#e2431e', '#f1ca3a']} 15
Php info scraped.
/entryinfo/3494-neste-oil-rally-finland-2012/161947/
Php foundhttps://www.ewrc-results.com/inc/json_driverinfo.php?event=3494&

In [4]:
df = pd.DataFrame(races)
df['Year'] = df['Year'].astype('int')
df_filtered = df[df['Year'] >= 2000]
df_filtered

,Driver,Name,URL,Year,Race_Result,DNG_Reason,Php_url,p_overall,stage,ssname
0,Thierry Neuville,Rally Sweden,/entryinfo/79247-rally-sweden-2023/3608291/,2023,3,Null,https://www.ewrc-results.com/inc/json_driverin...,5,5,SS1
1,Thierry Neuville,Rally Sweden,/entryinfo/79247-rally-sweden-2023/3608291/,2023,3,Null,https://www.ewrc-results.com/inc/json_driverin...,6,5,SS2
2,Thierry Neuville,Rally Sweden,/entryinfo/79247-rally-sweden-2023/3608291/,2023,3,Null,https://www.ewrc-results.com/inc/json_driverin...,6,6,SS3
3,Thierry Neuville,Rally Sweden,/entryinfo/79247-rally-sweden-2023/3608291/,2023,3,Null,https://www.ewrc-results.com/inc/json_driverin...,7,7,SS4
4,Thierry Neuville,Rally Sweden,/entryinfo/79247-rally-sweden-2023/3608291/,2023,3,Null,https://www.ewrc-results.com/inc/json_driverin...,6,6,SS5
...,...,...,...,...,...,...,...,...,...,...
2429,Thierry Neuville,RallyRACC Catalunya - Costa Daurada,/entryinfo/398-rallyracc-catalunya-costa-daura...,2009,DNF,[\r\n Alternato...,https://www.ewrc-results.com/inc/json_driverin...,21,25,SS9
2430,Thierry Neuville,RallyRACC Catalunya - Costa Daurada,/entryinfo/398-rallyracc-catalunya-costa-daura...,2009,DNF,[\r\n Alternato...,https://www.ewrc-results.com/inc/json_driverin...,21,22,SS10
2431,Thierry Neuville,RallyRACC Catalunya - Costa Daurada,/entryinfo/398-rallyracc-catalunya-costa-daura...,2009,DNF,[\r\n Alternato...,https://www.ewrc-results.com/inc/json_driverin...,21,23,SS11
2432,Thierry Neuville,RallyRACC Catalunya - Costa Daurada,/entryinfo/398-rallyracc-catalunya-costa-daura...,2009,DNF,[\r\n Alternato...,https://www.ewrc-results.com/inc/json_driverin...,21,20,SS12


In [5]:
df_filtered.to_csv('Thierry_Neuville.csv')